In [ ]:


spark = DatabricksSession.builder.profile("marvelmlops").getOrCreate()

df = spark.read.table("samples.nyctaxi.trips")
df.show(5)


In [1]:
# Cell to set up automatic reloading
%load_ext autoreload
%autoreload 2

In [3]:
import yaml
import logging

# Load configuration
with open('/Users/fcremer29/focus/marvelmlops/project-config.yml', 'r') as file:
    config = yaml.safe_load(file)

In [6]:
from src.packages.paths import AllPaths

ALL_PATHS = AllPaths(config=config)

In [9]:
ALL_PATHS.cremerf_catalog

'/Volumes/mlops_students/cremerfederico29/'

In [7]:
f'{ALL_PATHS.data_volume}/data.csv'

'/Volumes/mlops_students/cremerfederico29/data//data.csv'

In [56]:
from databricks.connect import DatabricksSession

df = spark.read.option("header",True).csv(f'{ALL_PATHS.data_volume}/hotel_reservations.csv').toPandas()


In [57]:
df.reset_index(drop=True)

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.5,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.8,1,Not_Canceled
36271,INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
36272,INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled
36273,INN36274,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,94.5,0,Canceled


In [58]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [37]:
print(f"Q de features numericas: {len(numeric_features)}")

Q de features numericas: 9


In [38]:
print(f"Q de features categoricas: {len(categorical_features)}")

Q de features categoricas: 8


In [39]:
print(f"Q de features totales: {len(df.columns)}")

Q de features totales: 19


In [59]:
# Define numeric and categorical features
numeric_features = [
    'no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights',
    'lead_time', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
    'avg_price_per_room', 'no_of_special_requests'
]
categorical_features = [
    'type_of_meal_plan', 'required_car_parking_space', 'room_type_reserved', 
    'arrival_year', 'arrival_month', 'arrival_date', 'market_segment_type', 
    'repeated_guest'
]

# Preprocessing for numeric data: convert data types and scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())  # Scale data
])

# Preprocessing for categorical data: fill missing values and apply one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Convert categorical data
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'  # This will drop other columns not listed explicitly
)

In [60]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [61]:
# Split data into features and target
X = df.drop('booking_status', axis=1)
y = df['booking_status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
spark.read.option("header",True).csv('/Volumes/mlops_students/cremerfederico29/data/hotel_reservations.csv').show()

+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------------------------------+------------------+----------------------+--------------+
|Booking_ID|no_of_adults|no_of_children|no_of_weekend_nights|no_of_week_nights|type_of_meal_plan|required_car_parking_space|room_type_reserved|lead_time|arrival_year|arrival_month|arrival_date|market_segment_type|repeated_guest|no_of_previous_cancellations|no_of_previous_bookings_not_canceled|avg_price_per_room|no_of_special_requests|booking_status|
+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------------------------------+--